In [ ]:
# Load data
%run 1.ReadingData.py

In [ ]:
INPUTS = x_train.shape[1]
OUTPUTS = t_train.shape[1]
NUM_TRAINING_EXAMPLES = int(round(x_train.shape[0]/1))
NUM_DEV_EXAMPLES = int (round (x_dev.shape[0]/1))
NUM_TEST_EXAMPLES = int (round (x_test.shape[0]/1))

In [ ]:
# Tensorboard
from keras.callbacks import TensorBoard
from time import time
NAME = "HousePricingClass-nn-30x2-{}".format(int(time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
# EarlyStopping
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', patience=10)

## HyperParameters

In [ ]:
n_epochs = 300
batch_size = 32
n_neurons_per_layer = [30, 30, 30]

In [ ]:
'''
examples = [1, 2, 3]
for example in examples:
    for l in range(example):
        print("Hola")
'''

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers

## GridSearch with TensorBoard Logs

In [ ]:
dense_layers = [3,4]
layer_sizes = [64]
learning_rates = [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for learning_rate in learning_rates:
        
            NAME = "{}-dense-{}-nodes-{}-learning_rate-{}".format(dense_layer, layer_size, learning_rate, int(time()))
            print(NAME)
            
            mlp = Sequential()
            
            mlp.add(Dense(layer_size, input_dim=INPUTS))
            mlp.add(Activation('relu'))
            
            for l in range(dense_layer-1):
                mlp.add(Dense(layer_size))
                mlp.add(Activation('relu'))
                
            # Final layer
            mlp.add(Dense(OUTPUTS))
            mlp.add(Activation('softmax'))
            
            tensorboard = TensorBoard(log_dir="LearnRate/{}".format(NAME))
            
            
            opt = optimizers.Adam(lr=learning_rate)
            mlp.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
            mlp.summary()    
            mlp.fit(x_train, t_train, batch_size=batch_size, epochs=n_epochs, verbose=2, validation_data=(x_dev, t_dev), callbacks=[tensorboard, earlystop])
            
            
        

## Architecture

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers

mlp = Sequential()

# Middle layers

# First
mlp.add(Dense(n_neurons_per_layer[0], input_dim=INPUTS))
mlp.add(Activation('relu'))
mlp.add(Dropout(0.2))

# Second
mlp.add(Dense(n_neurons_per_layer[1]))
mlp.add(Activation('relu'))
mlp.add(Dropout(0.2))

# Second
mlp.add(Dense(n_neurons_per_layer[2]))
mlp.add(Activation('relu'))
mlp.add(Dropout(0.2))

# Final layer
mlp.add(Dense(OUTPUTS))
mlp.add(Activation('softmax'))

opt = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
mlp.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
mlp.summary()

## Training

In [ ]:
data = mlp.fit(x_train, t_train, batch_size=batch_size, epochs=n_epochs, verbose=2, validation_data=(x_dev, t_dev), callbacks=[tensorboard])

## Performance Measures

In [ ]:
start = time()
loss, acc = mlp.evaluate(x_test, t_test, verbose=0)
end = time()
print('MLP took ' + str(end - start) + ' seconds')
print('Test loss: ' + str(loss) + ' - Accuracy: ' + str(acc))

In [ ]:
mlp.fit(x_train, t_train, batch_size=batch_size, epochs=n_epochs, verbose=2, validation_data=(x_dev, t_dev), callbacks=[tensorboard])